In [7]:
!pip3 install python-statemachine
%run helpers.ipynb
%run math_utils.ipynb

In [10]:
from statemachine import StateMachine, State
import numpy as np

In [6]:
class ADCS(StateMachine):
    """
    This class functions as the ADCS Controller.
    
    inputs = []
    outputs = []
    states = []
    
    transitions
    -----------
    
    """
    
    off = State(initial=True)
    detumbling = State()
    pointing = State()
    error = State()
    
    on_initialization = off.to(detumbling)
    to_pointing = detumbling.to(pointing)
    continuous_pointing = pointing.to(pointing)
    
    def __init__(self):
        self.moment = None
        self.pointing_vec = [0, 0, 0]
        super(ADCS, self).__init__()
        
    def b_dot(mag_field_vector, cang):
        detb = np.sqrt(np.dot(mag_field_vector, mag_field_vector))
        m = ((-1) / detb) * np.cross(mag_field_vector, cang)    
        return m

    def get_attitude_err(desired, estimated):

        delta = np.matmul(estimated, np.transpose(desired))
        attitude_err = -0.5 * np.array([
            delta[2, 1] - delta[1, 2],
            delta[0, 2] - delta[2, 0],
            delta[1, 0] - delta[0, 1],
        ])

        return attitude_err

    def get_pointing_torque(desired_pointing, actual_pointing):
        """Calculates the necessary control torques given input errors

            Args:
                attitude_err (numpy ndarray): the attitude error (3x1) of the
                    spacecraft at a given time
                attitude_rate_err (numpy ndarray): the attitude rate error (3x1) of
                    the spacecraft at a given time
                J (numpy ndarray): the spacecraft's inertia tensor (3x3) (kg * m^2)

            Returns:
                numpy ndarray: the control torque (3x1) produced by the PD
                    controller (N * m)
        """
        attitude_err = get_attitude_err(desired_pointing, actual_pointing)
        u = -np.matmul(np.diag([.1, .1, .1]), attitude_err)
        J = inertia(6, 0.34, 0.1, 0.1)
        return np.matmul(J, u)


    def get_pointing_moment(magnetic_field, desired, actual):
        moment = 1/(np.dot(magnetic_field)) * np.matmul(skew(magnetic_field), get_pointing_torque(desired, actual))
        return moment
    
    def before_pointing(self):
        
        # slow down rotation
        self.moment = b_dot()
        
    def after_pointing(self):
        
        # update pointing vector
        
        self.moment = get_pointing_moment()
        c1 = self.moment[0]
        c2 = self.moment[1]
        c3 = self.moment[2]
        
        # Check if greater
        
    
    def get_moment(self):
        return self.moment
    

class SafetyMonitor(StateMachine):
    
    safe = State(initial=True)
    error = State()
    
    to_error = safe.to(error, cond="moment_error")
    
    def moment_error(moment):
        return moment > 0.12
    
    def get_state(self):
        return self.current_state.id

IndentationError: expected an indented block (<ipython-input-6-f99f49d5ef8b>, line 97)